**Student: BJHZ7**

**10/01/2019 - Introduction to Programming**

# Final Assessment

Your university is collaborating with a company to develop a tool for processing reports of cases of a particular disease. The company has a very specific way of doing things, and has asked you to design and build a JavaScript object to interact with their proprietary code. Your supervisor wants you to submit an example of your code working within a Jupyter notebook (her preferred environment!) so she can review it before sending it off to the company.

Remember - your code will be tested against other input data, so make sure that you are being careful in your assumptions!

### Task 1

You have been provided with a sample of the case incidence records to help you test your code in the file "sampleCases.tsv". The company has done a questionable job of digitising these records, however, and you will need to clean them up a bit before you process them. Remember, .tsv files are "tab separated values", meaning that the entries are separated by tabs rather than commas (see [here](https://docs.python.org/3/reference/lexical_analysis.html#literals) for more information).

Read in the file and store each record as an element within a list.

In [ ]:
import numpy as np
import re
import matplotlib.pyplot as plt
from IPython.display import display, Javascript

In [ ]:
# This function is intended for locating the header line.

def letshandlethatfile(name):
    
    # Check if the type of the file provided is a tab separated file.
    
    if '.tsv' not in name:
        
        wrongname = True
        
        print('The file is not a tab separated file, please make sure it is!')
        print('')
        
        while(wrongname == True):
            
            newname = input('Please provide a tab separated file. Name of new file: ')
            
            if '.tsv' in newname:
                
                print('')
                print('A tab separated file has been provided! Let us continue.')
                print('')
                wrongname = False
                name = newname    
    
    yes = ['yes','Yes']
    no = ['no','No']
    
    accessmode = 'r'
    
    with open(name, accessmode) as Myfile:
        filetoprocess = [line.rstrip('\n').split('\t') for line in Myfile]
    
    #The file has now been read, but it can for many instances be of interest to know for sure which line is the header line.
    # We give the user the option to type in the location, if it is known in advance.
    
    print('The file has now been read, and it is time to locate the header')
    print('')
    answer = input('Do you know which line in the file, that is the header line? Answer: ')
    print('')
    
    if answer in yes:
        
        headerrow = input('Please type in the location of the header line by its line number. Location: ')
          
        if headerrow.isnumeric():
            
            # We verify that it is in fact the correct row that has been specified.
            
            headline,headerrow = confirmanswer(filetoprocess,headerrow)
        
        return filetoprocess, headerrow,headline,name
        
    elif answer in no:
        
        # If the user doesn't know the location, we print out the lines of the file, for the user to locate it.
        # This way of location the header line is of cause not optimal if you have many lines.
        # But if it was the case that the file contained many observations (lines) it is assumed that the header line
        # is known in advanced, alternatively an intelligent way of location the header line could be implemented but 
        # there is another time for that.
        
        print('Okay, we will locate it together then - no worries. Just tell me which line is the header line, based'+
             'on the output below')
        print('')
        l = 0
        
        for line in filetoprocess:
            print('%i: ' % (l),line)
            l += 1
        print('')
        
        headerrow = input('Please type in the location of the header line by its line number. Location provided: ')
               
        if headerrow.isnumeric():
            
            # We verify that it is in fact the correct row that has been specified.
            
            headline, headerrow = confirmanswer(filetoprocess,headerrow)
        
        return filetoprocess, headerrow,headline,name
        
    else:
        print('I dont get your answer, sorry. Please start over.')

In [ ]:
def confirmanswer(file,loc):
    
    yes = ['yes','Yes']
    no = ['no','No']
    
    print('')
    print('Chosen header line: ',file[int(loc)])
    print('')
    confirm = input('Is it the expected header line you have chosen? Answer: ')
    print('')

    if confirm in no:

        print('Try again then')
        print('')
        
        # While the user can't manage to provide the correct line, we let the user try again, and again and...
        
        while(confirm in no):

            loc = input('Please type in the location of the header line by its line number. Location provided: ')
            print('')
            
            if loc.isnumeric():

                print('Chosen header line: ',file[int(loc)])
                print('')
                
                confirm = input('Is it the expected header line you have chosen? Answer: ')
                
            else:
                print('')
                print('The chosen location is not numeric, so you will have to try again.')
                
        headline = file[int(loc)]
        
        return headline, loc
                
    elif confirm in yes:

        headline = file[int(loc)]
        
        return headline, loc

In [ ]:
records = letshandlethatfile('sampleCases.tsv')

In [ ]:
records

### Task 2

If you look at the records, you find that the latitudes and longitudes of cases are sometimes formatted strangely - apparently the automatic text recognitition software used by the company has a few bugs in it! Go through the records and prompt the user to input the corrected values in cases where the elements cannot be processed as numbers.

In [ ]:
# This function converts coordinates to numbers, either by input from the user or using regular expressions.

def processengine(row,pos,prompt=True):
    
    # Try to process the value as a number
    
    try:
                
        row[pos]=float(row[pos])

    except:

        # If an error occurs, continue to ask the user for input until a correct coordinate is provided.

        error = True

        # If the user wants to fill in the new value himself

        if prompt == True:

            while(error==True):
                
                # We show the user the value that couldn't be processed as a number.
                
                print('The value %s cannot be processed as a number.' % row[pos])
                print('')
                
                # The user gets the opportunity to correct the error.
                
                row[pos] = input('Please type a correct value. Value provided: ')
                
                # We run a check to see if the new value can be processed as a number.
                
                try:
                    row[pos] = float(row[pos])

                except:

                    print('The value %s cannot be processed as a number.' % row[pos])
                    print('')

                    row[pos] = input('Please type a correct value. Value provided: ')
                    
                    # We check the new input again
                    
                    try:
                        row[pos] = float(row[pos])
                    
                    except:
                
                        None
                
                # if the input can be processed as a number, we step of out the while loop.
                
                if type(row[pos])==float:
                    
                    error = False

        # Otherwise use regular expression to do the job

        else:
            
            # First we check if the coordinate is correctly separated (dot).
            
            if '.' in row[pos]:
                
                # The below regular expression remove everything there isn't a number and a dot.

                row[pos] = float(re.sub(r'[^0-9-.]',"",row[pos]))

            else:
                
                # If the coordinate, for what ever reason, isn't correctly separated, we replace the separated character
                # with a dot and continue

                temp = re.sub(r'[!|£|@|¤|%|&|/|(|)|€|{|[|]|}|#|¤|\^|\*|_|:|`|´|\+|\?| |;]',".",row[pos])

                if '.' in temp:
                    
                    # The below regular expression remove everything there isn't a number and a dot.

                    row[pos] = float(re.sub(r'[^0-9-.]',"",temp))

                else:
                    
                    # We end down here if the value is of a strange format, which could be missing or something like that.
                    
                    error = True
                    
                    while(error==True):
                        
                         # We show the user the value that appears strange.
                        
                        print('The value %s is strange, please type a new value' % temp)#row[pos]
                        print('')
                        
                        # The user gets the opportunity to correct the error.
                        
                        row[pos] = input('Please type a correct value. Value provided: ')
                        
                        # We run a check to see if the new value can be processed as a number.

                        try:
                            row[pos] = float(row[pos])

                        except:

                            print('The value %s cannot be processed as a number.' % row[pos])
                            print('')

                            row[pos] = input('Please type a correct value. Value provided: ')
                            
                            # We check the new input again
                    
                            try:
                                row[pos] = float(row[pos])

                            except:

                                None

                        # if the input can be processed as a number, we step of out the while loop.

                        if type(row[pos])==float:

                            error = False
    return row[pos]

In [ ]:
# This function converts the coordinates to numbers (floats) either manually by the user (prompt = True) or automatically 
# using regular expressions (prompt = False). The user can either specify the location of the latitude/longitude coordinates
# of they can be extracted using the header retreived in the previous task.

def letscleanthecoordinates(data,posheader=0,header='',poslat=0,poslng=0,prompt=True):
    
    # Removing the header line before we continue
    
    newdata = [sub[:] for sub in data]
    
    # It is assumed that the header is the first row, the user can specify if it is otherwise
    
    del newdata[posheader]
    
    # If no positions for the latitude and longitude columns are given
    
    if poslat == 0 and poslng == 0:
        
        # No positions should imply a header is present.
        
        if header != '':
            
            pos = 0
            
            # Locate the latitude/longitude columns in the provided header
            
            for head in header:
                
                if 'lat' in head:
                    
                    poslat = pos
                    
                if 'lon' in head:
                    
                    poslng = pos
                    
                pos += 1
            
            # The acutal check happens through the functions "processengine". The checking is about the value being 
            # a numeric value, and not for the value being in the correct range but that extent check is done in 
            # the Javascript object.
            
            for line in newdata:
           
                line[poslat] = processengine(line,poslat,prompt)
                line[poslng] = processengine(line,poslng,prompt)
            
            return newdata
            
        # If no positions are present nor header 
        
        else:
            
            print('Please start over and provide either positions of logitude/latitude columns or a header line')
    else:
        
        # If positions of latitude/longitude columns and header are provided
        
        if header != '':
            
            print('You have provided positions of latitude/longitude columns and a header line. ')
            print('Please only provide either positions or header line')
            
        # If only positions of longitude/latitude columns are provided 
        
        else:
            
            # The acutal check happens through the functions "processengine". The checking is about the value being 
            # a numeric value, and not for the value being in the correct range but that extent check is done in 
            # the Javascript object.
            
            for linee in newdata:
           
                linee[poslat] = processengine(linee,poslat,prompt)
                linee[poslng] = processengine(linee,poslng,prompt)
            
            return newdata

In [ ]:
# The locations of the lat/lon columns or the header found from the previous task can be provided for the calculations.
# Furthermore, the user has the possibility to go over the "wrong" coordinates manually, or let regular expressions do it.
# Either by setting prompt to True or False respectively.

newdata = letscleanthecoordinates(records[0],header=records[2], prompt = False)

In [ ]:
# Sanity check

newdata

### Task 3

It's time to format the data so that you can transfer it to the JavaScript portion of your work. You'll need to copy over the latitudes and longitudes. You may format these as a set of dictionaries, a pair of lists, or some other structure - it is up to you. 

In [ ]:
# The extraction could have been done in the two for-loops in the function "letscleanthecoordinates", especially
# because much of the same information is needed to extract the coordinates intelligently. 
# But for the sake of the third task, it is now done in a separate function.

def getthosepairs(data,posheader=0,header='', poslat=0,poslng=0,posid=0):
    
    # If no positions for the latitude and longitude columns are given
    if poslat == 0 and poslng == 0:
        
        # No positions should imply a header is present.
        if header != '':
            
            pos = 0
            
            # Locate the latitude/longitude/id columns in the provided header.
            
            for head in header:
                
                if 'lat' in head:
                    
                    poslat = pos
                    
                if 'lon' in head:
                    
                    poslng = pos
                    
                if 'id' in head:
                    
                    posid = pos
                    
                pos += 1
            
            # Extract the correctly formatted coordinates, ensured by the previous task, along with the id.
            # A list of dictonaries has been chosen as the preferred format
            
            coordinatepairs = []
            
            for line in newdata:
           
                coordinatepairs.append({'ID':line[posid],'LatCoor':line[poslat],'LngCoor':line[poslng]})
            
            return coordinatepairs
            
        # If no positions are present nor header 
        
        else:
            
            print('Please start over a provide either positions of logitude/latitude columns or a header line.')
            print('Please start over.')
    else:
        
        # If positions of latitude/longitude columns and header are provided
        
        if header != '':
            
            print('You have provided positions of latitude/longitude/ID columns or a header line. ')
            print('Please only provide either positions or header line.')
            print('Please start over.')
            
        # If only positions of longitude/latitude columns are provided 
        
        else:
            
            # Extract the correctly formatted coordinates, ensured by the previous task, along with the id.
            # A list of dictonaries has been chosen as the preferred format
            
            for line in newdata:
           
                coordinatepairs.append({'ID':line[posid],'LatCoor':line[poslat],'LngCoor':line[poslng]})
            
            return coordinatepairs
        

In [ ]:
pairs = getthosepairs(newdata,header = records[2])

In [ ]:
pairs

**We will now do some preperation in order to calculate the (weighted) centroid in the Javascript object**

**Note:** The below processing is done in Python since we are asked to pass the coordinates, not the entire data, to the Javascript object. There is no reason why the processing couldn't have been done in Javascript, and that would make sense if one wished to pass the entire data into the object, not just the coordinates. 

A centroid is the midpoint in a plane, and the (weighted) centroid is calculated in the following way, for the latitude and longitude coordinate respectively.

$Coordinate_{weighted} = \Big(\frac{\sum_{i=1}^n Connections_n\cdot Coordinate_n}{TotalConnections}\Big)$

The idea behind the formula is that observations with more connections, are assigned a larger weight and therefore are more central for the case study.
All observations are connected to itself, so the minimum connection is one, which ensures that all observations are included in the weighted centroid calculation.

The regular centroid follows the same formula, with all connections equal to one, implying that the coordinates of the centroid is the average coordinates of the observations.

So, if one observation is highly connected, one would expect to see the weighted centroid being dragged towards that observation.

In [ ]:
# lets extract the connection of each observation, in order to calculate the weighted centroid.

# Lets first get the note location in the header.

pos = 0

for step in records[2]:
    
    if 'note' in step:
    
        notepos = pos
    
    if 'id' in step:
        
        idpos = pos
    
    pos += 1

connections = []

for ele in newdata:
    
    # First we check if the observation at hand has any note about a potential connection, either because the note contains
    # the word "contact" or "case".
    
    if 'contact' in ele[4] or 'case' in ele[4]:
        
        # The regular expression extract everything that is a number.
        
        connections.append({'ID': ele[idpos],'Connected to': [int(x) for x in re.split(r'[^0-9]',ele[notepos]) if  x!= ''], 'Connections':1})
    
    # if the note doesn't contain information about any connections, a blank value is provided    
    
    else:
        
        connections.append({'ID': ele[idpos],'Connected to': [], 'Connections':1})

# Let's find how many observations that are connected to each observation

totalconnections = 0

for obs in np.arange(len(newdata)):
    
    for ele in connections[obs]['Connected to']:
        
        connections[ele-1]['Connections'] += 1
        
# Finally, count the total connections in the case study.

totalconnections = [x['Connections'] for x in connections]
totalconnections = sum(totalconnections)

# We now have everything for our Javascript object - yeeeha!

In [ ]:
# Check
print('The connections and number of connections for each observations are:')
print('')
[print(connect) for connect in connections]
print('')
print('The total connections are:',totalconnections)

### Task 4

The company wants you to create a kind of object called a `CaseStudy` to help them deal with all of the records associated with this file. In the future, they will use other `CaseStudy` objects to compare this set of data with other sets of data. But in this case, you just want to test creating a single `CaseStudy` and using its functions. 

The `CaseStudy` object should be able to:

* return its geographic extent (the minimum and maximum of both latitude and longitude) as a list (e.g. `[min_lat, max_lat, min_lon, max_lon]`)
* return the weighted centroid of the points as a list (e.g. `[cent_lat, cent_lon]`)

The `CaseStudy` object should also have attributes to hold:

* the name of the case study (a string)
* the year in which the case study was conducted (an integer)
* a value indicating whether the company took part in gathering the data (a boolean)

Your supervisor would like for you to create the instance of `CaseStudy` based on the data given here, calculate both the geographic extent and the weighted centroid, and export those two values back to your Python environment. You may set the name, year, and participation values of the `CaseStudy` to whatever you like.

In [ ]:
display(Javascript("""
///////////////////////////////////////////////////// Function library //////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
///////////////////////////////////////////////////// Error Checking functions //////////////////////////////////////////////////

///////////////////////////////////////////////////// Check the name provided /////////////////////////////////////////////////////

function checkname(name){
    // This function checks if the id only contains alphanumeric characters.
    // The function handles empty/undefined variables, spaces , no input ect.
  
    var check = new RegExp('^[a-zA-Z0-9]+$');
    var checkname=false,tempid;
    
    if (check.test(name)==true){
        
        // We are here if all characters are alphanumeric or undefined
        
        if (!!name==false){
            
            // we end here if the variable is undefined.
            // The two while loops below are identical and they give the user the option to reenter the id, and checks
            // that the new id only contains alphanumeric values and is not undefined.
            
            while(checkname==false){
                
                tempname = prompt('Error in establishing the ' + (CaseStudies.length+1) + 'th charging station. ' + 
                                'Please enter a ID containing only alphanumeric characters')
                
                if (check.test(tempname)==true){
                    
                    checkname=true;
                    
                    if (!!tempname==false){
                        
                        checkname=false;
                        
                    }
                }
            }
            // If the new id, provided by the user, only contains alphanumeric characters and is not undefined 
            
            return tempname
            
        } else {
            
            // if the variable only consists of alphanumeric characters and is not undefined
            
            return name
        }
        
    } else {
        
        // we end here if the initial id contains characters that aren't alphanumeric
        
        while(checkname==false){
            
            tempid = prompt('Error in establishing the ' + (CaseStudies.length+1) + 'th charging station. ' +
                            'Please enter a ID containing only alphanumeric characters')
            
            if (check.test(tempname)==true){
                
                checkname=true;
                
                if (!!tempname==false){
                    
                    checkname=false;
                    
                }
            }
        }
        
        return tempname
    }
}

///////////////////////////////////////////////////// Check the year ////////////////////////////////////////////////////////

function checkyear(year){
    // This function checks the requirement underlying the input for year dating, 
    // namely that it is a numeric integer value.
    
    if (Number.isInteger(year)==true){
        
        return year
        
    } else {
        
        var tempyear;
        
        while(Number.isInteger(Number(tempyear))==false){
            
            tempyear = prompt('Error in establishing the ' + (CaseStudies.length+1) + 'th charging station. ' +
                             'The capacity provided is not an integer, so please provide it again.')
            
        }
        
        return Number(tempyear)
        
    }
}

///////////////////////////////////////////////////// Check if the company took part in the data gathering /////////////////////////////////////////////////////

function checkgathering(gathered){
    // This function checks if the value for, whether the company took part in the gathering of the data
    // or not, is boolean or string with boolean value.
    
    var values = ['True','true','false','False'];
    var checkgath = false,tempgath;
    
    if (typeof(gathered)==='boolean' || (values.indexOf(gathered)>-1)){
        
        // We end here if the value is boolean or part of the 'string with boolean values' array (values)
        
        return gathered;
        
    } else {
        
        // we end here if the input is anything else than boolean or part of the 'string with boolean values' array (values)
        
        while(checkgath == false){
            
            tempgath = prompt('Error in establishing the ' + (CaseStudies.length+1) + 'th charging station. ' +
                              'Please enter whether a parking fee applies or not')
            
            if (typeof(tempgath)==='boolean' || (values.indexOf(tempgath)>-1)){
                
                checkgath = true;
            }
        }
        if (values.indexOf(tempgath)<2){
            
            return true;
            
        } else {
            
            return false;
            
        }
    }
}


///////////////////////////////////////////////////// Check the coordinates /////////////////////////////////////////////////////

function checkcoordinate(coordinate,min_coor,max_coor){
    // This function checks that the coordinates are of correct format. That implies checking that the values provided is 
    // non-empty, an integer and within the correct range, depending whether it is a x (longitude) or y (latitude) coordinate.
    // Longitude range: [-180,180] degrees
    // Latitude range: [-90,90] degrees
    
    var checkcoor = false,tempcoor,answer;
    var yes = ['yes', 'Yes'], no = ['no','No'];
    
    // The first thing to check is if the value is a numeric value, and lies in the correct span.
    
    if (!isNaN(coordinate) && (coordinate <= max_coor && coordinate >= min_coor)){
        
        // If that is the case, we check if it is an integer or a decimal value.
        
        if (Number.isInteger(coordinate)){
            
            // If it is an integer, we ask the user whether it is meant to be that way, or if it is an error.
            
            answer = prompt('Error in establishing the ' + (CaseStudies.length+1) + 'th charging station. ' +
                            'The numeric value you gave me is an integer, do you want me to continue with it (yes), or is it an error (no)?');
            
            if (yes.indexOf(answer)>-1){
                
                return coordinate;
                
            } else if (no.indexOf(answer)>-1) {
                
                // If it is an error, we ask the user to provide a new coordinate
                
                while(checkcoor==false){
                    
                    tempcoor = prompt('Error in establishing the ' + (CaseStudies.length+1) + 'th charging station. ' +
                                      'Please give me a new coordinate')
                    
                    // Again checking the format, as above
                    
                    if (!isNaN(tempcoor) && (tempcoor <= max_coor && tempcoor >= min_coor)){
                        
                        checkcoor = true;
                        
                        if (Number.isInteger(tempcoor)){
                            
                            console.log('Error in establishing the ' + (CaseStudies.length+1) + 'th charging station. ' +
                                        'Ouch, the coordinate is not a float - try again')
                            
                            checkcoor = false;
                        }
                    }
                }
                
                return Number(tempcoor)
                
            } else {
                
                // we end up down here, if the answer to the question concerning the integer value doesn't make sense.
                // we ask the user to provide input we understand.
                
                //console.log('I can not understand your answer.')
                
                while(checkcoor==false){
                    
                    // If the user can answers us clearly, we will ask the user to provide us with a new coordinate input.
                    
                    tempcoor = prompt('Error in establishing the ' + (CaseStudies.length+1) + 'th charging station. ' +
                                      'I can not understand your answer. Please give me a new coordinate. See the console for more information')
                    
                    // Checking the format again, as above
                    
                    if (!isNaN(tempcoor) && (tempcoor <= max_coor && tempcoor >= min_coor)){
                        
                        checkcoor = true;
                        
                        if (Number.isInteger(tempcoor)){
                            
                            console.log('Ouch, the coordinate is not a float - try again')
                            
                            checkcoor = false;
                            
                        }
                    }
                }
                
                return Number(tempcoor)
            }
                   
        }
        
    return coordinate
    
    } else {
        
        // we end up down here, if the input is not a numeric value e.g. a string, an empty variable ect.
        // we ask the user to provide input we understand.
        
        console.log('I can not understand the input for a coordinate you have provided me with. ' +
                   'Either you exceeded the limits for the coordinate or you gave me nonsense.')
        
        while(checkcoor==false){
            
            tempcoor = prompt('Error in establishing the ' + (CaseStudies.length+1) + 'th charging station. ' +
                              'Please give me a new coordinate, consult the console for more information.')
            
            //Checking the format of the input
            
            if (!isNaN(tempcoor) && (tempcoor <= max_coor && tempcoor >= min_coor)){
                
                checkcoor = true;
                
                if (Number.isInteger(tempcoor)){
                    
                    console.log('Ouch, the coordinate is not a float - try again')
                    
                    checkcoor = false;
                }
            }
        }
        
    return Number(tempcoor)
        
    }
}

///////////////////////////////////////////////////// Determining the Geographic Span /////////////////////////////////////////////////////

function geographicextent(array){
    // This function calculates the geographic span of the case study by locating the maximum and minimum
    // values of the coordinates
    
    var item;
    var lng_min = 180, lat_min = 90;
    var lng_max = -180, lat_max = -90;
    
    if (array.length>0){
        
        for (item of array){
            
            if (item.LatCoor > lat_max){
                
                lat_max = item.LatCoor
            }
            
            if (item.LatCoor < lat_min){
                
                lat_min = item.LatCoor
            }
            
            if (item.LngCoor > lng_max){
                
                lng_max = item.LngCoor
            }
            
            if (item.LngCoor < lng_min){
                
                lng_min = item.LngCoor
            }
        
        }
        
        //console.log('The maximum longitude coordinate is ' + lng_max);
        //console.log('The minimum longitude coordinate is ' + lng_min);
        //console.log('The maximum latitude coordinate is ' + lat_max);
        //console.log('The minimum latitude coordinate is ' + lat_min);
        
        return [lat_min,lat_max,lng_min,lng_max]
    
    } else {
        
        console.log('Error - no area defined');
    
    } 
}

///////////////////////////////////////////////////// Check the array of coordinates using the checkcoordinate function /////////////////////////////////////////////////////

function checklistofcoordinates(array){
    
    var item;
    
    for (item of array){
        
        item.LatCoor = checkcoordinate(item.LatCoor,-90,90);
        item.LngCoor = checkcoordinate(item.LngCoor,-180,180);
        
    }
    
    return array;
    
}

///////////////////////////////////////////////////// Calculating the centroid -  either with or without weights/////////////////////////////////////////////////////

function calculatecentroid(coordinates,weight = false,connections,total){
    
    var item;
    var pos = 0;
    var cen_lat = 0;
    var cen_lng = 0;
    
    // If not weights are wished
    
    if (weight == false){
        
        for (item of coordinates){
    
            cen_lat += item.LatCoor
            cen_lng += item.LngCoor
            pos += 1

        }
        
        return [cen_lat/pos,cen_lng/pos];
        
    // If weights are of interest
        
    } else if ( weight == true){
        
        for (item of coordinates){
    
            cen_lat += item.LatCoor*connections[pos]['Connections']
            cen_lng += item.LngCoor*connections[pos]['Connections']
            pos += 1

        }
        
        return [cen_lat/total,cen_lng/total];
        
    }
    
}

///////////////////////////////////////////////////// CaseStudy Object /////////////////////////////////////////////////////

function CaseStudy(name, year, gathered,coordinates,connections,total){
    
    this.name = checkname(name);
    this.year = checkyear(year);
    this.gathered = checkgathering(gathered);
    this.coordinates = checklistofcoordinates(coordinates);
    this.centroid = calculatecentroid(coordinates);
    this.weightedcentroid = calculatecentroid(coordinates,true,connections,total);
    this.extent = geographicextent(this.coordinates);
    this.report = function(){
        console.log('| Name of case study: ' + this.name,
                   '| Year: ' + this.year,
                   '| Did the company take part in the gathering of the data?: ' + this.gathered,
                   '| The geographic extent of the case study: ' + '['+this.extent+']',
                   '| The centroid of the case study: ' + '['+this.centroid+']',
                   '| The weighted centroid of the case study: ' + '['+this.weightedcentroid+']',
                   '|')
    }
    CaseStudies.push(this)
}

var CaseStudies = []

///////////////////////////////////////////////////// Create all the objects you need, I'll keep track of them ///////////////////

// Not giving any input allows the user the specify everything themselves - try it out!

var CaseStudy_1 = new CaseStudy('JohnDowStudy',2014,true,%s,%s,%s);

IPython.notebook.kernel.execute('CaseStudyExtent =' + CaseStudy_1.extent + ';');
IPython.notebook.kernel.execute('CaseStudyCentroid =' + CaseStudy_1.centroid + ';');
IPython.notebook.kernel.execute('CaseStudyWeightedCentroid =' + CaseStudy_1.weightedcentroid + ';');

console.log("");
console.log(CaseStudy_1);
console.log("");
CaseStudy_1.report();

""" % (pairs,connections,totalconnections)))

In [ ]:
# Just a sanity check

print('The geographic exent of the case study is:',CaseStudyExtent)
print('')
print('The coordinates of the centroid is:',CaseStudyCentroid)
print('')
print('The coordinates of the weighted centroid is:',CaseStudyWeightedCentroid)

In [ ]:
# Setting up for plottin the observations along with the weighted centroid.

lat_coor = []
lng_coor = []
id_coor = []
for subdict in pairs:
    lat_coor.append(subdict['LatCoor'])
    lng_coor.append(subdict['LngCoor'])
    id_coor.append(subdict['ID'])

In [ ]:
# Creating the plot of the observations and the weighted centroid.

fig = plt.figure()
ax = fig.add_subplot(111)

plt.plot(lat_coor,lng_coor,marker='o',linewidth=0)
pos = 0
for i in zip(lat_coor,lng_coor):
    ax.annotate('%s' %id_coor[pos], xy=i)
    pos += 1

# Cen = Centroid    
ax.plot(CaseStudyCentroid[0],CaseStudyCentroid[1],marker = 'o',linewidth=0)
ax.annotate('%s' %('Cen'), xy=CaseStudyCentroid)

# Weighted Centroid
ax.plot(CaseStudyWeightedCentroid[0],CaseStudyWeightedCentroid[1],marker = 'o',linewidth=0)
ax.annotate('%s' %('WC'), xy=CaseStudyWeightedCentroid)

plt.grid()
plt.show()

From the plot above, it os seen that the weighted centroid is closer to the second observation compared to the regular centroid. This is exactly as expected when the formula was proposed. 

### Task 5

As a final step, write out the geographic extent, centroid, and the original filename of the data to a file named "output.txt".

In [ ]:
filename = 'output.txt'
accessmode = 'w'

with open(filename, accessmode) as Myfile:
    Myfile.write('The name of the file for the investigated case study is: '+ records[3] +'\n')
    Myfile.write('The geographic extent of the investigated case study is: '+ str(CaseStudyExtent)+'\n')
    Myfile.write('The coordinates of the centroid in the investigated case study is: '+ str(CaseStudyCentroid)+'\n')
    Myfile.write('The coordinates of the weighted centroid in the investigated case study is: '+ str(CaseStudyWeightedCentroid))